## Import packages

In [ ]:
import sys
from pathlib import Path

# Get the parent directory of the current notebook
notebook_dir = Path.cwd()  # Folder containing the notebook
project_root = notebook_dir.parent  # Go up one level to 'project/'

# Add the 'src/' folder to Python's search path
sys.path.append(str(project_root / "src"))
sys.path.append(str(project_root / "data"))

# Verify
print(sys.path)

import yfinance as yf
from fredapi import Fred
import pandas as pd 
from dotenv import load_dotenv
import os
import os

## API Abrufen

In [ ]:
# Load environment variables from .env file
load_dotenv()
fred_api_key = os.getenv("FRED_API_KEY")

print(fred_api_key)
# Federal Reserve Bank of ST.Louis API
fred = Fred(api_key=fred_api_key)


In [ ]:
 # Datumintervall festlegen
start = '2010-01-01'
end = '2025-06-22'

def interpolate_to_daily(df, method='linear'):
    df = df.copy()
    df.index = pd.to_datetime(df.index)
    daily_index = pd.date_range(start=start, end=end, freq='D')
    df = df.reindex(daily_index)
    df_interpolated = df.interpolate(method=method)
    df_interpolated.index.name = 'Date'
    return df_interpolated







### Get date range

In [ ]:
full_date_range=pd.date_range(start=start,end=end,freq="D")

len(full_date_range)

## Extract Macroeconomic Factors

*USD_Index (dxy)*

In [ ]:
usd_index_df = yf.download('DX-Y.NYB', start=start, end=end)['Close']
usd_index_df.name =  'USD_Index'

freq = pd.infer_freq(usd_index_df.index)
print(f"Inferred frequency: {freq}") # Daily Frequency (Buissness days, excluding weekends and holidays)

usd_index_df_interpolated = interpolate_to_daily(usd_index_df, 'pad')
nan_count_per_column = usd_index_df_interpolated.isna().sum()
first_valid_value = usd_index_df_interpolated['DX-Y.NYB'].dropna().iloc[0]
usd_index_df_interpolated['DX-Y.NYB'].fillna(first_valid_value, inplace=True)
usd_index_df_interpolated.columns=['USD_Index']
usd_index_df_interpolated


*EUR_USD (eurusd)*

In [ ]:
eur_usd_df = yf.download('EURUSD=X', start=start, end=end)['Close']

freq = pd.infer_freq(eur_usd_df.index)
print(f"Inferred frequency: {freq}") # Daily Frequency (Buissness days, excluding weekends and holidays)

eur_usd_df_interpolated = interpolate_to_daily(eur_usd_df, 'pad')
nan_count_per_column = eur_usd_df_interpolated.isna().sum()
first_valid_value = eur_usd_df_interpolated['EURUSD=X'].dropna().iloc[0]
eur_usd_df_interpolated['EURUSD=X'].fillna(first_valid_value, inplace=True)
eur_usd_df_interpolated.columns=['EUR_USD']
eur_usd_df_interpolated

*SP500 (sp500)*

In [ ]:
sp_500_df = yf.download('^GSPC', start=start, end=end)['Close']
sp_500_df.name =  'SP500'

freq = pd.infer_freq(sp_500_df.index)
print(f"Inferred frequency: {freq}") # Daily Frequency (Buissness days, excluding weekends and holidays)

sp_500_df_interpolated = interpolate_to_daily(sp_500_df, 'pad')
nan_count_per_column = sp_500_df_interpolated.isna().sum()
first_valid_value = sp_500_df_interpolated['^GSPC'].dropna().iloc[0]
sp_500_df_interpolated['^GSPC'].fillna(first_valid_value, inplace=True)
sp_500_df_interpolated.columns=['SP_500']
sp_500_df_interpolated

*Oil_Price (Oil)*

In [ ]:
oil_df = yf.download('CL=F', start=start, end=end)['Close']

freq = pd.infer_freq(oil_df.index)
print(f"Inferred frequency: {freq}") # Daily Frequency (Buissness days, excluding weekends and holidays)

oil_df_interpolated = interpolate_to_daily(oil_df, 'pad')
nan_count_per_column = oil_df_interpolated.isna().sum()
first_valid_value = oil_df_interpolated['CL=F'].dropna().iloc[0]
oil_df_interpolated['CL=F'].fillna(first_valid_value, inplace=True)
oil_df_interpolated.columns=['Oil']
oil_df_interpolated

*Bitcoin_Price (bitcoin)*


In [ ]:
bitcoin_df = yf.download('BTC-USD', start=start, end=end)['Close']
bitcoin_df.name = 'Bitcoin_Price'

freq = pd.infer_freq(bitcoin_df.index)
print(f"Inferred frequency: {freq}") # Daily Frequency

bitcoin_df_interpolated = interpolate_to_daily(bitcoin_df, 'pad')
bitcoin_df_interpolated.columns=['Bitcoin']
bitcoin_df_interpolated


*VIX_Index (vix)*

In [ ]:
vix_index_df = yf.download('^VIX', start=start, end=end)['Close']
freq = pd.infer_freq(vix_index_df.index)
print(f"Inferred frequency: {freq}") # Daily Frequency (Buissness days, excluding weekends and holidays)

vix_index_df_interpolated = interpolate_to_daily(vix_index_df, 'pad')
nan_count_per_column = vix_index_df_interpolated.isna().sum()
first_valid_value = vix_index_df_interpolated['^VIX'].dropna().iloc[0]
vix_index_df_interpolated['^VIX'].fillna(first_valid_value, inplace=True)
vix_index_df_interpolated.columns=['Vix']
vix_index_df_interpolated

*CPI (cpi)*

In [ ]:
cpi_df = fred.get_series('CPIAUCNS', observation_start=start, observation_end=end).to_frame(name='CPI')

freq = pd.infer_freq(cpi_df.index)
print(f"Inferred frequency: {freq}") # Monthly Frequency

cpi_df_interpolated = interpolate_to_daily(cpi_df)
cpi_df_interpolated.columns=['CPI']
cpi_df_interpolated

*Unemployment_rate (unemp)*

In [ ]:
unemp_df = fred.get_series('UNRATE', observation_start=start, observation_end=end).to_frame(name='Unemployment_Rate')

freq = pd.infer_freq(unemp_df.index)
print(f"Inferred frequency: {freq}") # Monthly Frequency

unemp_df_interpolated = interpolate_to_daily(unemp_df)
unemp_df_interpolated

*GDP (gdp)*

In [ ]:
gdp_df = fred.get_series('GDP', observation_start=start, observation_end=end).to_frame(name='GPD')

freq = pd.infer_freq(gdp_df.index)
print(f"Inferred frequency: {freq}") # Quartal Frequency

gdp_df_interpolated = interpolate_to_daily(gdp_df)
gdp_df_interpolated

*Intrest_Rate (interest)*

In [ ]:
interest_df = fred.get_series('FEDFUNDS', observation_start=start, observation_end=end).to_frame(name='US_Interest_Rate')

freq = pd.infer_freq(interest_df.index)
print(f"Inferred frequency: {freq}") # Quartal Frequency

interest_df_interpolated = interpolate_to_daily(interest_df)
interest_df_interpolated

*Zinsen (FEDFUNDS)*

In [ ]:
zinsen_df = fred.get_series('FEDFUNDS', observation_start=start, observation_end=end).to_frame(name='Zinsen')

freq = pd.infer_freq(zinsen_df.index)
print(f"Inferred frequency: {freq}") # Monthly Frequency

zinsen_df_interpolated = interpolate_to_daily(zinsen_df)
zinsen_df_interpolated

*Gold-ETF-Bestände*

In [ ]:
gold_etf_df = fred.get_series('GVZCLS', observation_start=start, observation_end=end).to_frame(name='Gold_ETFs')

freq = pd.infer_freq(gold_etf_df.index)
print(f"Inferred frequency: {freq}") # Monthly Frequency

gold_etf_df_interpolated = interpolate_to_daily(gold_etf_df)
gold_etf_df_interpolated

*M2-Geldmenge (M2SL, MYAGM2EZM196N)*

In [ ]:
m2_us_df = fred.get_series('M2SL', observation_start=start, observation_end=end).to_frame(name='M2_US')
m2_euro_df = fred.get_series('MYAGM2EZM196N', observation_start=start, observation_end=end).to_frame(name='M2_Euro')

freq_u2_us_df= pd.infer_freq(m2_us_df.index)
print(f"Inferred frequency u2_us_df : {freq_u2_us_df}") # Monthly Frequency

freq_m2_euro_df= pd.infer_freq(m2_euro_df.index)
print(f"Inferred frequency m2_euro_df: {freq_m2_euro_df}") # Monthly Frequency

m2_us_df_interpolated = interpolate_to_daily(m2_us_df)
print(m2_us_df_interpolated)
m2_euro_df_interpolated = interpolate_to_daily(m2_euro_df)
m2_euro_df_interpolated

*Geopolitische Spannungen (Kriege, Konflikte) (GPR Index)*

In [ ]:
GPR_ind=pd.read_excel("../data/raw/data_gpr_daily_recent.xls")
GPR=GPR_ind[['date','GPRD']]
GPR['Date']=pd.to_datetime(GPR['date'])
GPR = GPR[(GPR['Date']>=start) & (GPR['Date']<=end)]

GPR=GPR.set_index(GPR['Date'])
GPR_interpolated=interpolate_to_daily(GPR)
GPR_interpolated=GPR_interpolated.drop(columns=['Date','date'])
GPR_interpolated

Gold

In [ ]:
gold_df = yf.download('GC=F', start=start, end=end)['Close']

freq = pd.infer_freq(gold_df.index)
print(f"Inferred frequency: {freq}") # Daily Frequency (Buissness days, excluding weekends and holidays)

gold_df_interpolated = interpolate_to_daily(gold_df, 'pad')
nan_count_per_column = gold_df_interpolated.isna().sum()
first_valid_value = gold_df_interpolated['GC=F'].dropna().iloc[0]
gold_df_interpolated['GC=F'].fillna(first_valid_value, inplace=True)
gold_df_interpolated.columns=['Gold']
gold_df_interpolated

## Combine all factors into a single dataset

In [ ]:
df_fin= pd.concat([gold_df_interpolated,
                   usd_index_df_interpolated,
                   eur_usd_df_interpolated,
                   sp_500_df_interpolated,
                   oil_df_interpolated,
                   bitcoin_df_interpolated,
                   vix_index_df_interpolated,
                   cpi_df_interpolated,
                   unemp_df_interpolated,
                   gdp_df_interpolated,
                   interest_df_interpolated,
                   zinsen_df_interpolated,
                   gold_etf_df_interpolated,
                   m2_euro_df_interpolated,
                   m2_us_df_interpolated,
                   GPR_interpolated
                   ],axis=1)

df_fin

#### Check NaNs values

In [ ]:
df_fin.isna().sum()

In [ ]:
df_fin['Gold'].tail(16)

## Export DataFrame as .parquet

In [ ]:
df_fin.to_parquet("../data/processed/gold_macro_combined.parquet")